## File list division

In [1]:
import random

filelist_path = 'filelists/filelist.txt'
trn_filels_path = 'filelists/train_filelist.txt'
val_filels_path = 'filelists/val_filelist.txt'
val_ratio = 0.1
filels_by_chara = {}

with open(filelist_path, 'r', encoding='utf8') as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip().split('\t')
        sid = line[1]
        if sid not in filels_by_chara:
            filels_by_chara[sid] = [line]
        else:
            filels_by_chara[sid].append(line)
            
val_filewrite_num_ls = [int(len(filels_by_chara[k]) * val_ratio) for k in filels_by_chara.keys()]

with open(val_filels_path, 'w', encoding='utf8') as f:
    for idx, i in enumerate(val_filewrite_num_ls):
        for line in random.sample(filels_by_chara[str(idx)], i):
            f.writelines(["\t".join(line) + "\n"])
            filels_by_chara[str(idx)].remove(line)

with open(trn_filels_path, 'w', encoding='utf8') as f:
    for idx in range(len(val_filewrite_num_ls)):
        f.writelines(["\t".join(line) + "\n" for line in filels_by_chara[str(idx)]])

## File list preprosess

In [11]:
import argparse
import text
from utils import load_filepaths_and_text

# parser = argparse.ArgumentParser()
# parser.add_argument("--out_extension", default="cleaned")
# parser.add_argument("--text_index", default=2, type=int)
# parser.add_argument("--filelists", nargs="+", default=["filelists/train_filelist.txt", "filelists/val_filelist.txt"])
# parser.add_argument("--text_cleaners", nargs="+", default=["chinese_cleaners2"])

args = {"out_extension": "cleaned",
        "text_index": 2,
        "filelists": ["filelists/train_filelist.txt", "filelists/val_filelist.txt"],
        "text_cleaners": ["chinese_cleaners2"]}
    
for filelist in args.filelists:
    print("START:", filelist)
    filepaths_and_text = load_filepaths_and_text(filelist)
    for i in range(len(filepaths_and_text)):
        original_text = filepaths_and_text[i][args.text_index]
        cleaned_text = text._clean_text(original_text, args.text_cleaners)
        filepaths_and_text[i][args.text_index] = cleaned_text

    new_filelist = filelist + "." + args.out_extension
    with open(new_filelist, "w", encoding="utf-8") as f:
        f.writelines(["\t".join(x) + "\n" for x in filepaths_and_text])

AttributeError: 'dict' object has no attribute 'filelists'

## Audio resampling

In [6]:
import librosa
import os
import soundfile as sf
def resample_rate(path, new_sample_rate = 22050):
    files = os.listdir(path)
    for file in files:
        file_path = path + file
        signal, sr = librosa.load(file_path, sr=48000)
        new_signal = librosa.resample(signal, sr, new_sample_rate)
        sf.write(file_path, new_signal , new_sample_rate)
resample_rate('./src/audio/')